In [31]:
import pyarrow as pa
import pyarrow.feather as feather
import pyarrow.parquet as pq
import pyarrow.compute as pc
import numpy as np
from pathlib import Path
from rich.progress import track

In [2]:
!ls -l /data/aiomics/massspec_cache/uniprot/cho/*.parquet
!ls -l /data/aiomics/massspec_cache/uniprot/cho/*.arrow


-rw-rw-r--. 1 djs10 645div   257977993 Apr 20 11:51 /data/aiomics/massspec_cache/uniprot/cho/cho_uniprot_tryptic_2.parquet
-rw-rw-r--. 1 djs10 645div   283786246 Apr 20 12:19 /data/aiomics/massspec_cache/uniprot/cho/cho_uniprot_tryptic_2_tmt.parquet
-rw-r--r--. 1 djs10 645div 12269375854 May 25 14:13 /data/aiomics/massspec_cache/uniprot/cho/predicted_cho_uniprot_tryptic_2_0.parquet
-rw-r--r--. 1 djs10 645div 12573970866 May 25 14:17 /data/aiomics/massspec_cache/uniprot/cho/predicted_cho_uniprot_tryptic_2_1.parquet
-rw-r--r--. 1 djs10 645div 12844369481 May 25 14:21 /data/aiomics/massspec_cache/uniprot/cho/predicted_cho_uniprot_tryptic_2_2.parquet
-rw-r--r--. 1 djs10 645div 12310726608 May 25 14:26 /data/aiomics/massspec_cache/uniprot/cho/predicted_cho_uniprot_tryptic_2_3.parquet
-rw-r--r--. 1 djs10 645div 11927726552 May 25 14:30 /data/aiomics/massspec_cache/uniprot/cho/predicted_cho_uniprot_tryptic_2_4.parquet
-rw-r--r--. 1 djs10 645div  9185134284 May 25 14:33 /data/aiomics/massspec_

In [17]:
for input in track(Path('/data/search_asms2023/cho').glob('*.arrow')):
    output = input.with_suffix('.parquet')
    table = feather.read_table(input)
    pq.write_table(table, output)

Output()

In [19]:
table = pq.read_table("/data/aiomics/massspec_cache/uniprot/cho/predicted_cho_uniprot_tryptic_2_0.parquet")
table

pyarrow.Table
id: uint64
instrument: string
instrument_type: string
instrument_model: string
ion_mode: string
ionization: string
name: string
synonyms: string
scan: string
collision_energy: double
retention_time: double
collision_gas: string
insource_voltage: int64
sample_inlet: string
ev: double
nce: double
charge: int8
precursor_mz: double
exact_mass: double
exact_mw: double
annotations: large_list<item: struct<ion_type: dictionary<values=string, indices=int32, ordered=0>, product_charge: int8, isotope: uint8, ion_subtype: dictionary<values=string, indices=int32, ordered=0>, position: uint16, end_position: uint16, aa_before: dictionary<values=string, indices=int32, ordered=0>, aa_after: dictionary<values=string, indices=int32, ordered=0>, ptm_before: dictionary<values=string, indices=int32, ordered=0>, ptm_after: dictionary<values=string, indices=int32, ordered=0>>>
  child 0, item: struct<ion_type: dictionary<values=string, indices=int32, ordered=0>, product_charge: int8, isotope: u

In [49]:
query_table = pq.read_table("/home/djs10/gitlab/masskit/masskit_ext/build/release/src/search/test.parquet")
query_table

pyarrow.Table
id: uint64
charge: int8
ev: double
instrument: string
instrument_type: string
instrument_model: string
ion_mode: string
ionization: string
name: string
synonyms: string
scan: string
nce: double
collision_energy: double
retention_time: double
collision_gas: string
insource_voltage: int64
sample_inlet: string
intensity: large_list<item: double>
  child 0, item: double
stddev: large_list<item: double>
  child 0, item: double
product_massinfo: struct<tolerance: double, tolerance_type: dictionary<values=string, indices=int32, ordered=0>, mass_type: dictionary<values=string, indices=int32, ordered=0>, neutral_loss: string, neutral_loss_charge: int64, evenly_spaced: bool>
  child 0, tolerance: double
  child 1, tolerance_type: dictionary<values=string, indices=int32, ordered=0>
  child 2, mass_type: dictionary<values=string, indices=int32, ordered=0>
  child 3, neutral_loss: string
  child 4, neutral_loss_charge: int64
  child 5, evenly_spaced: bool
mz: large_list<item: double>


In [50]:
mods = query_table["mod_names"]
np_mods = mods.combine_chunks().to_numpy(zero_copy_only=False)
mask = pa.array(~np.array([ np.any(x==737) for x in np_mods ]))

In [56]:
new_table = query_table.filter(mask)
pq.write_table(new_table,"/home/djs10/gitlab/masskit/masskit_ext/build/release/src/search/test_filtered.parquet")

In [54]:
len(query_table)

143044

In [55]:
len(new_table)

116548